In [ ]:
!rm -rf /kaggle/working/tracking_results

In [ ]:
pip install ultralytics

## Prepare the dataset for YOLOv8

In [ ]:
import os
import shutil
import yaml
from pathlib import Path
import numpy as np
from tqdm import tqdm
from PIL import Image

# Define the paths
DATASET_PATH = "/kaggle/input/cv-multiobject-dectection-dataset"
VISDRONE_MOT_TRAIN = os.path.join(DATASET_PATH, "VisDrone2019-MOT-train/VisDrone2019-MOT-train")
VISDRONE_MOT_VAL = os.path.join(DATASET_PATH, "VisDrone2019-MOT-val/VisDrone2019-MOT-val")
VISDRONE_MOT_TEST = os.path.join(DATASET_PATH, "VisDrone2019-MOT-test-challenge/VisDrone2019-MOT-test-challenge")
OUTPUT_PATH = "/kaggle/working/yolo_dataset"

# Create output directories
os.makedirs(OUTPUT_PATH, exist_ok=True)
os.makedirs(os.path.join(OUTPUT_PATH, "images", "train"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_PATH, "images", "val"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_PATH, "images", "test"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_PATH, "labels", "train"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_PATH, "labels", "val"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_PATH, "labels", "test"), exist_ok=True)

# VisDrone MOT categories
# The VisDrone annotation format appears to be:
# <frame_index>,<target_id>,<bbox_left>,<bbox_top>,<bbox_width>,<bbox_height>,<score>,<object_category>,<truncation>,<occlusion>
#
# The object_category is defined as:
# 0: ignored regions
# 1: pedestrian
# 2: people
# 3: bicycle
# 4: car
# 5: van
# 6: truck
# 7: tricycle
# 8: awning-tricycle
# 9: bus
# 10: motor
# 11: others

# Define the class mapping - we'll map VisDrone classes to YOLOv8 format
# We'll skip class 0 (ignored regions) and class 11 (others)
class_mapping = {
    1: 0,  # pedestrian
    2: 1,  # people
    3: 2,  # bicycle
    4: 3,  # car
    5: 4,  # van
    6: 5,  # truck
    7: 6,  # tricycle
    8: 7,  # awning-tricycle
    9: 8,  # bus
    10: 9,  # motor
}

# Class names for YAML file
class_names = [
    'pedestrian',
    'people',
    'bicycle',
    'car',
    'van',
    'truck',
    'tricycle',
    'awning-tricycle',
    'bus',
    'motor'
]

def convert_annotation(annotation_file, sequence_folder, split_type):
    """
    Convert VisDrone annotation format to YOLO format
    """
    sequence_name = os.path.basename(sequence_folder)
    
    # Read the annotation file
    with open(annotation_file, 'r') as f:
        lines = f.readlines()
    
    # Group annotations by frame
    frame_annotations = {}
    for line in lines:
        line = line.strip().split(',')
        frame_idx = int(line[0])
        if frame_idx not in frame_annotations:
            frame_annotations[frame_idx] = []
        frame_annotations[frame_idx].append(line)
    
    # Process each frame
    for frame_idx, annotations in frame_annotations.items():
        # Source image path
        img_file = f"{frame_idx:07d}.jpg"
        img_path = os.path.join(sequence_folder, img_file)
        
        if not os.path.exists(img_path):
            continue
        
        # Destination paths
        dst_img_path = os.path.join(OUTPUT_PATH, "images", split_type, f"{sequence_name}_{img_file}")
        dst_label_path = os.path.join(OUTPUT_PATH, "labels", split_type, f"{sequence_name}_{img_file.replace('.jpg', '.txt')}")
        
        # Copy the image
        shutil.copy(img_path, dst_img_path)
        
        # Convert annotations to YOLO format and write to file
        with open(dst_label_path, 'w') as f:
            # Get image dimensions
            # We'll use PIL to get the image dimensions
            from PIL import Image
            img = Image.open(img_path)
            img_width, img_height = img.size
            
            for anno in annotations:
                obj_id = int(anno[1])
                bbox_left = float(anno[2])
                bbox_top = float(anno[3])
                bbox_width = float(anno[4])
                bbox_height = float(anno[5])
                obj_class = int(anno[7])
                
                # Skip ignored regions and others
                if obj_class == 0 or obj_class == 11:
                    continue
                
                # Map to YOLO class
                if obj_class not in class_mapping:
                    continue
                yolo_class = class_mapping[obj_class]
                
                # Convert bbox to YOLO format (x_center, y_center, width, height) - normalized
                x_center = (bbox_left + bbox_width / 2) / img_width
                y_center = (bbox_top + bbox_height / 2) / img_height
                width = bbox_width / img_width
                height = bbox_height / img_height
                
                # Write to file
                f.write(f"{yolo_class} {x_center} {y_center} {width} {height}\n")

def process_dataset():
    """
    Process the dataset using the existing train/val/test splits
    """
    # Process training data
    train_sequences_folder = os.path.join(VISDRONE_MOT_TRAIN, "sequences")
    train_annotations_folder = os.path.join(VISDRONE_MOT_TRAIN, "annotations")
    
    train_sequences = [f for f in os.listdir(train_sequences_folder) if os.path.isdir(os.path.join(train_sequences_folder, f))]
    
    for seq in tqdm(train_sequences, desc="Processing train set"):
        seq_path = os.path.join(train_sequences_folder, seq)
        anno_path = os.path.join(train_annotations_folder, f"{seq}.txt")
        if os.path.exists(anno_path):
            convert_annotation(anno_path, seq_path, "train")
    
    # Process validation data
    val_sequences_folder = os.path.join(VISDRONE_MOT_VAL, "sequences")
    val_annotations_folder = os.path.join(VISDRONE_MOT_VAL, "annotations")
    
    val_sequences = [f for f in os.listdir(val_sequences_folder) if os.path.isdir(os.path.join(val_sequences_folder, f))]
    
    for seq in tqdm(val_sequences, desc="Processing validation set"):
        seq_path = os.path.join(val_sequences_folder, seq)
        anno_path = os.path.join(val_annotations_folder, f"{seq}.txt")
        if os.path.exists(anno_path):
            convert_annotation(anno_path, seq_path, "val")
    
    # Process test data
    test_sequences_folder = os.path.join(VISDRONE_MOT_TEST, "sequences")
    test_annotations_folder = os.path.join(VISDRONE_MOT_TEST, "annotations")
    
    test_sequences = [f for f in os.listdir(test_sequences_folder) if os.path.isdir(os.path.join(test_sequences_folder, f))]
    
    for seq in tqdm(test_sequences, desc="Processing test set"):
        seq_path = os.path.join(test_sequences_folder, seq)
        # For test-challenge, we might not have annotations
        anno_path = os.path.join(test_annotations_folder, f"{seq}.txt")
        if os.path.exists(anno_path):
            convert_annotation(anno_path, seq_path, "test")
        else:
            # If no annotations, just copy images
            for img_file in os.listdir(seq_path):
                if img_file.endswith('.jpg'):
                    src_img_path = os.path.join(seq_path, img_file)
                    dst_img_path = os.path.join(OUTPUT_PATH, "images", "test", f"{seq}_{img_file}")
                    shutil.copy(src_img_path, dst_img_path)

def create_yaml_file():
    """
    Create YAML configuration file for YOLOv8
    """
    yaml_content = {
        'path': OUTPUT_PATH,
        'train': os.path.join('images', 'train'),
        'val': os.path.join('images', 'val'),
        'test': os.path.join('images', 'test'),
        'nc': len(class_names),  # number of classes
        'names': class_names
    }
    
    with open(os.path.join(OUTPUT_PATH, 'visdrone.yaml'), 'w') as f:
        yaml.dump(yaml_content, f, default_flow_style=False)

if __name__ == "__main__":
    print("Starting dataset preparation...")
    process_dataset()
    create_yaml_file()
    
    # Print some statistics
    train_images = len(os.listdir(os.path.join(OUTPUT_PATH, "images", "train")))
    val_images = len(os.listdir(os.path.join(OUTPUT_PATH, "images", "val")))
    test_images = len(os.listdir(os.path.join(OUTPUT_PATH, "images", "test")))
    
    print(f"Dataset preparation complete!")
    print(f"Train images: {train_images}")
    print(f"Validation images: {val_images}")
    print(f"Test images: {test_images}")
    print(f"YAML configuration file created at: {os.path.join(OUTPUT_PATH, 'visdrone.yaml')}")

## Configure YOLOv8 model

In [ ]:
import yaml
from ultralytics import YOLO
import os

# Set the paths
OUTPUT_PATH = "/kaggle/working/yolo_dataset"
MODELS_PATH = "/kaggle/working/models"
YAML_PATH = os.path.join(OUTPUT_PATH, "visdrone.yaml")

# Create models directory if it doesn't exist
os.makedirs(MODELS_PATH, exist_ok=True)

def select_model_size():
    """
    Select the appropriate model size based on your requirements
    - nano: Fastest but least accurate (ideal for resource-constrained environments)
    - small: Good balance between speed and accuracy
    - medium: More accurate but slower than small
    - large: Most accurate but slowest
    """
    model_sizes = {
        'n': 'yolov8n',  # Nano
        's': 'yolov8s',  # Small
        'm': 'yolov8m',  # Medium
        'l': 'yolov8l',  # Large
        'x': 'yolov8x'   # Extra Large
    }
    
    print("Select YOLOv8 model size:")
    print("n - Nano (fastest, least accurate)")
    print("s - Small (good balance)")
    print("m - Medium (more accurate, slower)")
    print("l - Large (most accurate, slowest)")
    print("x - Extra Large (highest accuracy, very slow)")
    
    choice = input("Enter your choice (default: s): ").lower() or 's'
    if choice not in model_sizes:
        print(f"Invalid choice: {choice}. Using Small model.")
        choice = 's'
    
    model_name = model_sizes[choice]
    print(f"Selected model: {model_name}")
    return model_name

def load_config():
    """Load the dataset configuration from YAML file"""
    with open(YAML_PATH, 'r') as f:
        config = yaml.safe_load(f)
    return config

def configure_model(model_size):
    """Configure the YOLOv8 model with the selected size"""

    model_path = os.path.join(MODELS_PATH, f"{model_size}.pt")
    
    # Check if the model file exists, otherwise download it
    if not os.path.exists(model_path):
        print(f"🔽 {model_size}.pt not found. Downloading it now...")
        model = YOLO(model_size)  # This will automatically download the model
        model.export(format="torch")  # Save it in the correct format
        os.rename(f"{model_size}.pt", model_path)  # Move to desired location
        print(f"✅ Model downloaded and saved at {model_path}")
    else:
        print(f"✅ Model found at {model_path}")

    # Load the pre-trained model
    model = YOLO(model_path)
    
    # Model configuration
    model_config = {
        'task': 'detect',
        'model': model_size,
        'data': YAML_PATH,
        'epochs': 100,
        'patience': 10,
        'batch': 16,
        'imgsz': 640,
        'device': 0,  # Use GPU if available
        'workers': 8,
        'project': MODELS_PATH,
        'name': f'visdrone_{model_size.lower()}',
    }
    
    # Save the configuration
    config_path = os.path.join(MODELS_PATH, f"{model_size.lower()}_config.yaml")
    with open(config_path, 'w') as f:
        yaml.dump(model_config, f)
    
    print(f"📄 Model configuration saved to {config_path}")
    return model_config

def main():
    """Main function to configure the YOLOv8 model"""
    dataset_config = load_config()
    print(f"Dataset configuration loaded from {YAML_PATH}")
    print(f"Number of classes: {dataset_config['nc']}")
    print(f"Classes: {dataset_config['names']}")
    
    model_size = select_model_size()
    model_config = configure_model(model_size)
    
    print("\nModel configuration complete!")
    print("Next steps:")
    print("Run the training script")
    print("Monitor training metrics")
    print("Evaluate model performance")
    
    return model_config

if __name__ == "__main__":
    model_config = main()

## GPU

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Should say Tesla T4


In [ ]:
import ultralytics
print(f"Ultralytics YOLOv8 version: {ultralytics.__version__}")

from ultralytics.utils.torch_utils import select_device
device = select_device('')
print(f"YOLOv8 Selected Device: {device}")

In [ ]:
import torch
import ultralytics
from ultralytics.utils.torch_utils import select_device

print(f"Ultralytics YOLOv8 version: {ultralytics.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU Name: {torch.cuda.get_device_name(0)}")
print(f"YOLOv8 Selected Device: {select_device('')}")

## Train the model

In [ ]:
# Train YOLOv8 Model
import os
import yaml
import torch
from ultralytics import YOLO
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Set the paths
MODELS_PATH = "/kaggle/working/models"
OUTPUT_PATH = "/kaggle/working/yolo_dataset"
YAML_PATH = os.path.join(OUTPUT_PATH, "visdrone.yaml")

def load_model_config(config_path=None):
    """Load model configuration from file or use default"""
    if config_path and os.path.exists(config_path):
        with open(config_path, 'r') as f:
            config = yaml.safe_load(f)
        return config
    
    # Default configuration if no file is provided
    config_files = [f for f in os.listdir(MODELS_PATH) if f.endswith('_config.yaml')]
    if config_files:
        config_path = os.path.join(MODELS_PATH, config_files[0])
        with open(config_path, 'r') as f:
            config = yaml.safe_load(f)
        return config
    
    # Fallback to very basic configuration
    return {
        'task': 'detect',
        'model': 'YOLOv8s.pt',
        'data': YAML_PATH,
        'epochs': 3,
        'patience': 10,
        'batch': 16,
        'imgsz': 640,
        'device': 0,
        'workers': 8,
        'project': MODELS_PATH,
        'name': 'visdrone_default',
    }

def check_gpu_availability():
    """Check if GPU is available and print info"""
    if torch.cuda.is_available():
        device_count = torch.cuda.device_count()
        print(f"GPU available! Found {device_count} GPU(s).")
        for i in range(device_count):
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"Current GPU: {torch.cuda.current_device()}")
        print(f"Memory allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
        print(f"Memory cached: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
    else:
        print("No GPU available. Training will be slow on CPU.")

def custom_hyperparameters():
    """Customize hyperparameters for training"""
    hyperparams = {}
    
    # Learning rate
    hyperparams['lr0'] = 0.01  # Initial learning rate
    hyperparams['lrf'] = 0.01  # Final learning rate ratio
    
    # Optimizer parameters
    hyperparams['momentum'] = 0.937
    hyperparams['weight_decay'] = 0.0005
    
    # Augmentation parameters
    hyperparams['hsv_h'] = 0.015  # HSV-Hue augmentation
    hyperparams['hsv_s'] = 0.7    # HSV-Saturation augmentation
    hyperparams['hsv_v'] = 0.4    # HSV-Value augmentation
    hyperparams['degrees'] = 0.0   # Rotation augmentation
    hyperparams['translate'] = 0.1  # Translation augmentation
    hyperparams['scale'] = 0.5     # Scale augmentation
    hyperparams['fliplr'] = 0.5    # Horizontal flip augmentation
    hyperparams['mosaic'] = 1.0    # Mosaic augmentation
    
    return hyperparams

def train_model(config, hyperparams=None):
    """Train the YOLOv8 model with the given configuration"""

    # Ensure we train for only 3 epochs
    config['epochs'] = 3  

    # Explicitly set device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    # Load model
    model = YOLO(config['model'])

    # Train the model
    results = model.train(
        data=config['data'],
        epochs=config['epochs'],
        patience=config['patience'],
        batch=config['batch'],
        imgsz=config['imgsz'],
        device=device,  # Ensure GPU is used
        workers=config['workers'],
        project=config['project'],
        name=config['name'],
        exist_ok=True,
        pretrained=True,
        **(hyperparams or {})
    )
    
    return model, results


def plot_training_results(results_file):
    """Plot training metrics from results CSV"""
    if not os.path.exists(results_file):
        print(f"Results file not found: {results_file}")
        return
    
    # Load results
    results = pd.read_csv(results_file)
    
    # Plot metrics
    metrics = ['box_loss', 'cls_loss', 'dfl_loss', 'precision', 'recall', 'mAP50', 'mAP50-95']
    
    plt.figure(figsize=(20, 14))
    
    for i, metric in enumerate(metrics):
        if metric in results.columns:
            plt.subplot(3, 3, i+1)
            plt.plot(results['epoch'], results[metric], 'b-')
            plt.title(f'Training {metric}')
            plt.xlabel('Epoch')
            plt.ylabel(metric)
            plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(os.path.join(MODELS_PATH, 'training_metrics.png'))
    plt.show()

def main():
    """Main function to train the YOLOv8 model"""
    # Check GPU availability
    check_gpu_availability()
    
    # Load model configuration
    config_files = [f for f in os.listdir(MODELS_PATH) if f.endswith('_config.yaml')]
    if config_files:
        config_path = os.path.join(MODELS_PATH, config_files[0])
        print(f"Loading configuration from {config_path}")
        config = load_model_config(config_path)
    else:
        print("No configuration file found. Using default configuration.")
        config = load_model_config()
    
    # Customize hyperparameters
    hyperparams = custom_hyperparameters()
    
    # Train the model
    model, results = train_model(config, hyperparams)
    
    # Plot training results
    results_file = os.path.join(MODELS_PATH, config['name'], 'results.csv')
    plot_training_results(results_file)
    
    print("\nTraining complete!")
    print(f"Model saved at: {os.path.join(MODELS_PATH, config['name'])}")
    print("Next steps:")
    print("1. Evaluate model performance")
    print("2. Run inference on test images")
    print("3. Export model for deployment")
    
    return model

if __name__ == "__main__":
    model = main()

## Detection

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# Load your trained model
model_path = "/kaggle/working/models/visdrone_yolov8s/weights/best.pt"
model = YOLO(model_path)

# Path to the test image
image_path = "/kaggle/input/cv-multiobject-dectection-dataset/VisDrone2019-MOT-train/VisDrone2019-MOT-train/sequences/uav0000013_00000_v/0000001.jpg"

# Run inference on the image
results = model.predict(image_path, conf=0.25)

# Plot the results
plt.figure(figsize=(12, 10))
img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
result_img = results[0].plot()
plt.imshow(result_img)
plt.axis('off')
plt.title("Prediction Results")
plt.show()

# Print detection statistics
print(f"Number of detections: {len(results[0].boxes)}")
print(f"Detected classes: {results[0].names}")

# Print detailed results for top 5 detections
for i, box in enumerate(results[0].boxes[:5]):
    class_id = int(box.cls)
    confidence = float(box.conf)
    x1, y1, x2, y2 = box.xyxy[0].tolist()
    print(f"Detection {i+1}: Class: {results[0].names[class_id]}, Confidence: {confidence:.2f}, Coordinates: [{int(x1)}, {int(y1)}, {int(x2)}, {int(y2)}]")

## Evaluate the model

In [ ]:
# Evaluate YOLOv8 Model
import os
import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO
import seaborn as sns
from sklearn.metrics import confusion_matrix
import torch
import cv2

# Set the paths
MODELS_PATH = "/kaggle/working/models"
OUTPUT_PATH = "/kaggle/working/yolo_dataset"
YAML_PATH = os.path.join(OUTPUT_PATH, "visdrone.yaml")
RESULTS_PATH = os.path.join(OUTPUT_PATH, "evaluation_results")

# Create results directory if it doesn't exist
os.makedirs(RESULTS_PATH, exist_ok=True)

def load_model():
    """Load the trained model"""
    # Find the latest model weights
    model_runs = [d for d in os.listdir(MODELS_PATH) if os.path.isdir(os.path.join(MODELS_PATH, d))]
    if not model_runs:
        print("No trained models found. Please train a model first.")
        return None
    
    latest_run = max(model_runs, key=lambda x: os.path.getmtime(os.path.join(MODELS_PATH, x)))
    weights_path = os.path.join(MODELS_PATH, latest_run, "weights", "best.pt")
    
    if not os.path.exists(weights_path):
        weights_path = os.path.join(MODELS_PATH, latest_run, "weights", "last.pt")
    
    if not os.path.exists(weights_path):
        print("No model weights found. Please train a model first.")
        return None
    
    print(f"Loading model from {weights_path}")
    model = YOLO(weights_path)
    return model

def load_class_names():
    """Load class names from the YAML file"""
    with open(YAML_PATH, 'r') as f:
        config = yaml.safe_load(f)
    return config['names']

def evaluate_model(model):
    """Evaluate the model on the validation dataset"""
    if model is None:
        return None
    
    print("Evaluating model on validation dataset...")
    val_path = os.path.join(OUTPUT_PATH, "images", "val")
    
    # Run validation
    results = model.val(
        data=YAML_PATH,
        split='val',
        batch=16,
        imgsz=640,
        conf=0.25,
        iou=0.5,
        max_det=300,
        save_json=True,
        save_hybrid=True,
        plots=True
    )
    
    return results

def plot_confusion_matrix(model, class_names):
    """Plot confusion matrix from model evaluation"""
    conf_matrix_path = os.path.join(model.trainer.save_dir, "confusion_matrix.png")
    
    if not os.path.exists(conf_matrix_path):
        print("Confusion matrix not found. Generating...")
        # Generate confusion matrix from model predictions
        results = model.val(data=YAML_PATH, split='val', conf=0.25, iou=0.5)
        
    if os.path.exists(conf_matrix_path):
        # Copy the confusion matrix to results directory
        import shutil
        shutil.copy(conf_matrix_path, os.path.join(RESULTS_PATH, "confusion_matrix.png"))
        print(f"Confusion matrix saved to {os.path.join(RESULTS_PATH, 'confusion_matrix.png')}")
        
        # Display confusion matrix
        plt.figure(figsize=(12, 10))
        img = cv2.imread(conf_matrix_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img)
        plt.axis('off')
        plt.title("Confusion Matrix")
        plt.tight_layout()
        plt.savefig(os.path.join(RESULTS_PATH, "confusion_matrix_display.png"))
        plt.show()
    else:
        print("Confusion matrix could not be generated.")

def plot_precision_recall_curve(model):
    """Plot precision-recall curve from model evaluation"""
    pr_curve_path = os.path.join(model.trainer.save_dir, "PR_curve.png")
    
    if os.path.exists(pr_curve_path):
        # Copy the PR curve to results directory
        import shutil
        shutil.copy(pr_curve_path, os.path.join(RESULTS_PATH, "PR_curve.png"))
        print(f"Precision-Recall curve saved to {os.path.join(RESULTS_PATH, 'PR_curve.png')}")
        
        # Display PR curve
        plt.figure(figsize=(12, 10))
        img = cv2.imread(pr_curve_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img)
        plt.axis('off')
        plt.title("Precision-Recall Curve")
        plt.tight_layout()
        plt.savefig(os.path.join(RESULTS_PATH, "PR_curve_display.png"))
        plt.show()
    else:
        print("Precision-Recall curve not found.")

def plot_f1_confidence_curve(model):
    """Plot F1-Confidence curve from model evaluation"""
    f1_curve_path = os.path.join(model.trainer.save_dir, "F1_curve.png")
    
    if os.path.exists(f1_curve_path):
        # Copy the F1 curve to results directory
        import shutil
        shutil.copy(f1_curve_path, os.path.join(RESULTS_PATH, "F1_curve.png"))
        print(f"F1-Confidence curve saved to {os.path.join(RESULTS_PATH, 'F1_curve.png')}")
        
        # Display F1 curve
        plt.figure(figsize=(12, 10))
        img = cv2.imread(f1_curve_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img)
        plt.axis('off')
        plt.title("F1-Confidence Curve")
        plt.tight_layout()
        plt.savefig(os.path.join(RESULTS_PATH, "F1_curve_display.png"))
        plt.show()
    else:
        print("F1-Confidence curve not found.")

def calculate_performance_metrics(results):
    """Calculate and print performance metrics"""
    if results is None:
        print("No results available for performance metrics calculation.")
        return
    
    # Extract metrics from results
    metrics = {
        'mAP50': float(results.box.map50),  # Convert tensor to float
        'mAP50-95': float(results.box.map),  # Convert tensor to float
        'Precision': float(results.box.p.mean()),  # Mean precision
        'Recall': float(results.box.r.mean()),  # Mean recall
        'F1-Score': float((2 * results.box.p * results.box.r / (results.box.p + results.box.r + 1e-16)).mean())  # Mean F1-score
    }
    
    # Print metrics
    print("\nPerformance Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")
    
    # Save metrics to CSV
    metrics_df = pd.DataFrame([metrics])
    metrics_df.to_csv(os.path.join(RESULTS_PATH, "performance_metrics.csv"), index=False)
    print(f"Performance metrics saved to {os.path.join(RESULTS_PATH, 'performance_metrics.csv')}")
    
    # Plot metrics
    plt.figure(figsize=(10, 6))
    sns.barplot(x=list(metrics.keys()), y=list(metrics.values()))
    plt.title('Model Performance Metrics')
    plt.ylim(0, 1)
    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_PATH, "performance_metrics.png"))
    plt.show()
    
    return metrics

def analyze_class_performance(results, class_names):
    """Analyze performance per class"""
    if results is None or not hasattr(results, 'names'):
        print("No results available for class performance analysis.")
        return
    
    # Extract class metrics
    class_metrics = {}
    for i, name in enumerate(class_names):
        if i < len(results.box.ap50):
            class_metrics[name] = {
                'AP50': results.box.ap50[i].item(),
                'Precision': results.box.p[i].item() if isinstance(results.box.p, torch.Tensor) and len(results.box.p) > i else 0,
                'Recall': results.box.r[i].item() if isinstance(results.box.r, torch.Tensor) and len(results.box.r) > i else 0
            }
    
    # Create DataFrame
    class_df = pd.DataFrame(class_metrics).T
    
    # Print class metrics
    print("\nClass Performance:")
    print(class_df)
    
    # Save class metrics to CSV
    class_df.to_csv(os.path.join(RESULTS_PATH, "class_performance.csv"))
    print(f"Class performance metrics saved to {os.path.join(RESULTS_PATH, 'class_performance.csv')}")
    
    # Plot class metrics
    plt.figure(figsize=(12, 8))
    class_df.plot(kind='bar')
    plt.title('Performance Metrics by Class')
    plt.xlabel('Class')
    plt.ylabel('Score')
    plt.ylim(0, 1)
    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_PATH, "class_performance.png"))
    plt.show()
    
    return class_df

def main():
    """Main function to evaluate the model"""
    model = load_model()
    if model is None:
        return
    
    class_names = load_class_names()
    
    # Evaluate model
    results = evaluate_model(model)
    
    # Calculate performance metrics
    metrics = calculate_performance_metrics(results)
    
    # Analyze class performance
    class_df = analyze_class_performance(results, class_names)
    
    # Plot confusion matrix
    plot_confusion_matrix(model, class_names)
    
    # Plot precision-recall curve
    plot_precision_recall_curve(model)
    
    # Plot F1-confidence curve
    plot_f1_confidence_curve(model)
    print("\nModel evaluation complete!")
    print(f"All evaluation results saved to {RESULTS_PATH}")
    
    return results, metrics, class_df

if __name__ == "__main__":
    results, metrics, class_df = main()

## Inference and deployment

In [ ]:
# Inference and Deployment for YOLOv8 Model
import os
import yaml
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from PIL import Image
import torch
import time
import glob
from tqdm import tqdm

# Set the paths
MODELS_PATH = "/kaggle/working/models"
OUTPUT_PATH = "/kaggle/working/yolo_dataset"
YAML_PATH = os.path.join(OUTPUT_PATH, "visdrone.yaml")
INFERENCE_PATH = os.path.join(OUTPUT_PATH, "inference_results")
TEST_IMAGES_PATH = os.path.join(OUTPUT_PATH, "images", "test")
EXPORT_PATH = os.path.join(OUTPUT_PATH, "exported_models")

# Create results directory if it doesn't exist
os.makedirs(INFERENCE_PATH, exist_ok=True)
os.makedirs(EXPORT_PATH, exist_ok=True)

def load_model():
    """Load the trained model"""
    # Find the latest model weights
    model_runs = [d for d in os.listdir(MODELS_PATH) if os.path.isdir(os.path.join(MODELS_PATH, d))]
    if not model_runs:
        print("No trained models found. Please train a model first.")
        return None
    
    latest_run = max(model_runs, key=lambda x: os.path.getmtime(os.path.join(MODELS_PATH, x)))
    weights_path = os.path.join(MODELS_PATH, latest_run, "weights", "best.pt")
    
    if not os.path.exists(weights_path):
        weights_path = os.path.join(MODELS_PATH, latest_run, "weights", "last.pt")
    
    if not os.path.exists(weights_path):
        print("No model weights found. Please train a model first.")
        return None
    
    print(f"Loading model from {weights_path}")
    model = YOLO(weights_path)
    return model

def load_class_names():
    """Load class names from the YAML file"""
    with open(YAML_PATH, 'r') as f:
        config = yaml.safe_load(f)
    return config['names']

def run_inference_on_image(model, image_path, class_names, conf_threshold=0.25, save_result=True):
    """Run inference on a single image"""
    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Could not read image: {image_path}")
        return None
    
    # Run inference
    start_time = time.time()
    results = model.predict(image, conf=conf_threshold, iou=0.5, max_det=300, verbose=False)
    inference_time = time.time() - start_time
    
    # Process results
    result = results[0]
    
    # Get detections
    boxes = result.boxes.xyxy.cpu().numpy()
    scores = result.boxes.conf.cpu().numpy()
    class_ids = result.boxes.cls.cpu().numpy().astype(int)
    
    # Draw detections
    result_image = image.copy()
    for box, score, class_id in zip(boxes, scores, class_ids):
        x1, y1, x2, y2 = box.astype(int)
        label = f"{class_names[class_id]}: {score:.2f}"
        
        # Generate a color based on class_id
        color = (int(hash(class_names[class_id]) % 255), 
                 int(hash(class_names[class_id] * 2) % 255), 
                 int(hash(class_names[class_id] * 3) % 255))
        
        # Draw rectangle and text
        cv2.rectangle(result_image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(result_image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    # Save result
    if save_result:
        base_name = os.path.basename(image_path)
        output_path = os.path.join(INFERENCE_PATH, f"result_{base_name}")
        cv2.imwrite(output_path, result_image)
        print(f"Inference result saved to {output_path}")
    
    return result_image, inference_time, len(boxes)

def run_inference_on_test_set(model, class_names, conf_threshold=0.25, max_images=20):
    """Run inference on multiple test images"""
    print("Running inference on test images...")
    
    # Get test images
    test_images = glob.glob(os.path.join(TEST_IMAGES_PATH, "*.jpg"))
    if not test_images:
        print("No test images found.")
        return
    
    # Limit number of images to process
    test_images = test_images[:max_images]
    
    # Run inference on each image
    inference_times = []
    detection_counts = []
    
    for img_path in tqdm(test_images):
        result_image, inference_time, detection_count = run_inference_on_image(
            model, img_path, class_names, conf_threshold
        )
        inference_times.append(inference_time)
        detection_counts.append(detection_count)
    
    # Calculate statistics
    avg_time = np.mean(inference_times)
    avg_detections = np.mean(detection_counts)
    
    print(f"\nInference on {len(test_images)} test images complete.")
    print(f"Average inference time: {avg_time:.4f} seconds per image")
    print(f"Average number of detections: {avg_detections:.2f}")
    
    # Create inference report
    report = {
        "num_images": len(test_images),
        "avg_inference_time": avg_time,
        "avg_detections": avg_detections,
        "confidence_threshold": conf_threshold
    }
    
    # Save report
    import json
    with open(os.path.join(INFERENCE_PATH, "inference_report.json"), "w") as f:
        json.dump(report, f, indent=4)
    
    return report

def run_inference_from_webcam(model, class_names, conf_threshold=0.25):
    """Run inference on webcam (for demonstration purposes)"""
    print("Starting webcam inference...")
    
    # Check if webcam exists
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Could not open webcam. Skipping webcam inference.")
        return
    
    # Set up window
    cv2.namedWindow("YOLOv8 Inference", cv2.WINDOW_NORMAL)
    
    try:
        while True:
            # Read frame
            ret, frame = cap.read()
            if not ret:
                break
            
            # Run inference
            results = model.predict(frame, conf=conf_threshold, iou=0.5, max_det=300, verbose=False)
            result = results[0]
            
            # Get detections
            boxes = result.boxes.xyxy.cpu().numpy()
            scores = result.boxes.conf.cpu().numpy()
            class_ids = result.boxes.cls.cpu().numpy().astype(int)
            
            # Draw detections
            for box, score, class_id in zip(boxes, scores, class_ids):
                x1, y1, x2, y2 = box.astype(int)
                label = f"{class_names[class_id]}: {score:.2f}"
                
                # Generate a color based on class_id
                color = (int(hash(class_names[class_id]) % 255), 
                        int(hash(class_names[class_id] * 2) % 255), 
                        int(hash(class_names[class_id] * 3) % 255))
                
                # Draw rectangle and text
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            
            # Show frame
            cv2.imshow("YOLOv8 Inference", frame)
            
            # Break loop if 'q' pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        # Release resources
        cap.release()
        cv2.destroyAllWindows()

def export_model(model, format="onnx"):
    """Export the model to the specified format"""
    print(f"Exporting model to {format} format...")
    
    # Supported formats
    supported_formats = ["onnx", "torchscript", "openvino", "coreml", "tflite"]
    
    if format not in supported_formats:
        print(f"Unsupported format: {format}. Supported formats: {supported_formats}")
        return
    
    # Export model
    try:
        model.export(format=format, imgsz=640)
        
        # Move exported model to EXPORT_PATH
        source_path = os.path.join(os.path.dirname(model.ckpt_path), f"{os.path.basename(model.ckpt_path).split('.')[0]}.{format}")
        if os.path.exists(source_path):
            import shutil
            dest_path = os.path.join(EXPORT_PATH, f"yolov8_visdrone.{format}")
            shutil.copy(source_path, dest_path)
            print(f"Model exported to {dest_path}")
    except Exception as e:
        print(f"Error exporting model: {e}")

def export_all_formats(model):
    """Export the model to all supported formats"""
    formats = ["onnx", "torchscript", "openvino", "tflite"]
    for format in formats:
        export_model(model, format)

def main():
    """Main function for inference and deployment"""
    model = load_model()
    if model is None:
        return
    
    class_names = load_class_names()
    
    # Run inference on test set
    report = run_inference_on_test_set(model, class_names, conf_threshold=0.25, max_images=20)
    
    # Export model
    while True:
        print("\nChoose model export format:")
        print("1. ONNX (recommended for deployment)")
        print("2. TorchScript")
        print("3. OpenVINO")
        print("4. TensorFlow Lite")
        print("5. Export all formats")
        print("6. Skip export")
        
        choice = input("Enter your choice (default: 1): ") or "1"
        
        if choice == "1":
            export_model(model, "onnx")
            break
        elif choice == "2":
            export_model(model, "torchscript")
            break
        elif choice == "3":
            export_model(model, "openvino")
            break
        elif choice == "4":
            export_model(model, "tflite")
            break
        elif choice == "5":
            export_all_formats(model)
            break
        elif choice == "6":
            print("Skipping model export.")
            break
        else:
            print("Invalid choice. Please try again.")
    
    print("\nInference and deployment complete!")
    print(f"All inference results saved to {INFERENCE_PATH}")
    print(f"Exported models saved to {EXPORT_PATH}")
    
    return model, report

if __name__ == "__main__":
    model, report = main()

In [ ]:
import matplotlib.pyplot as plt
import os
from glob import glob
from PIL import Image

# Path to the folder containing images
image_folder = "/kaggle/working/yolo_dataset/inference_results/"

# Get all image file paths
image_paths = sorted(glob(os.path.join(image_folder, "*.jpg")))  # Change "*.jpg" if images have a different extension

# Plot images
plt.figure(figsize=(15, 15))
columns = 4
rows = (len(image_paths) + columns - 1) // columns  # Calculate number of rows dynamically

for i, image_path in enumerate(image_paths, 1):
    img = Image.open(image_path)
    plt.subplot(rows, columns, i)
    plt.imshow(img)
    plt.axis("off")
    plt.title(os.path.basename(image_path))

plt.tight_layout()
plt.show()

## DeepSort

In [ ]:
pip install ultralytics torch opencv-python numpy tqdm supervision

In [ ]:
pip install lap

In [ ]:
import os
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from tqdm import tqdm
import lap  # Linear assignment problem solver

class KalmanFilter:
    """Simple Kalman Filter implementation for tracking"""
    def __init__(self):
        # State transition matrix
        self.F = np.array([[1, 0, 0, 0, 1, 0, 0, 0],
                           [0, 1, 0, 0, 0, 1, 0, 0],
                           [0, 0, 1, 0, 0, 0, 1, 0],
                           [0, 0, 0, 1, 0, 0, 0, 1],
                           [0, 0, 0, 0, 1, 0, 0, 0],
                           [0, 0, 0, 0, 0, 1, 0, 0],
                           [0, 0, 0, 0, 0, 0, 1, 0],
                           [0, 0, 0, 0, 0, 0, 0, 1]])
        
        # Measurement matrix
        self.H = np.array([[1, 0, 0, 0, 0, 0, 0, 0],
                           [0, 1, 0, 0, 0, 0, 0, 0],
                           [0, 0, 1, 0, 0, 0, 0, 0],
                           [0, 0, 0, 1, 0, 0, 0, 0]])
        
        # Measurement noise
        self.R = np.eye(4) * 0.1
        
        # Process noise
        self.Q = np.eye(8) * 0.1
        self.Q[4:, 4:] *= 10
        
        # Error covariance
        self.P = np.eye(8) * 10
        
        # State
        self.x = np.zeros((8, 1))
        
    def predict(self):
        """Predict next state"""
        self.x = self.F @ self.x
        self.P = self.F @ self.P @ self.F.T + self.Q
        return self.x[:4].flatten()
        
    def update(self, z):
        """Update state with measurement z"""
        z = z.reshape(-1, 1)
        y = z - self.H @ self.x
        S = self.H @ self.P @ self.H.T + self.R
        K = self.P @ self.H.T @ np.linalg.inv(S)
        self.x = self.x + K @ y
        I = np.eye(8)
        self.P = (I - K @ self.H) @ self.P
        return self.x[:4].flatten()
        
    def initiate(self, measurement):
        """Initialize state with first measurement"""
        self.x = np.zeros((8, 1))
        self.x[:4, 0] = measurement
        return self.x[:4].flatten()


class Track:
    """Track class for DeepSORT"""
    count = 0
    
    def __init__(self, detection, class_id):
        self.id = Track.count
        Track.count += 1
        
        self.class_id = class_id
        self.hits = 1
        self.age = 1
        self.time_since_update = 0
        
        # Initialize Kalman filter
        self.kf = KalmanFilter()
        
        # Initialize state
        box = detection[:4]
        self.mean = self.kf.initiate(box)
        
        # Detection confidence
        self.confidence = detection[4] if len(detection) > 4 else 1.0
        
    def predict(self):
        """Predict next state"""
        self.mean = self.kf.predict()
        self.age += 1
        self.time_since_update += 1
        return self.mean
        
    def update(self, detection):
        """Update track with new detection"""
        box = detection[:4]
        self.mean = self.kf.update(box)
        self.hits += 1
        self.time_since_update = 0
        
        # Update confidence
        if len(detection) > 4:
            self.confidence = detection[4]
        
        # Update class_id if provided
        if len(detection) > 5:
            self.class_id = detection[5]
        
        return self.mean
        
    def get_state(self):
        """Get current state"""
        return self.mean
    
    def is_confirmed(self):
        """Check if track is confirmed"""
        return self.hits >= 3
    
    def is_deleted(self):
        """Check if track should be deleted"""
        return self.time_since_update > 30


class SimpleTracker:
    """Simple implementation of DeepSORT tracker"""
    def __init__(self):
        self.tracks = []
        self.max_iou_distance = 0.7
        self.max_age = 30
        self.min_hits = 3
        
    def update(self, detections):
        """Update tracks with new detections"""
        # Predict locations of existing tracks
        for track in self.tracks:
            track.predict()
        
        # Match detections to tracks
        matches, unmatched_tracks, unmatched_detections = self._match(detections)
        
        # Update matched tracks
        for track_idx, detection_idx in matches:
            self.tracks[track_idx].update(detections[detection_idx])
        
        # Mark unmatched tracks
        for track_idx in unmatched_tracks:
            self.tracks[track_idx].time_since_update += 1
        
        # Add new tracks
        for detection_idx in unmatched_detections:
            detection = detections[detection_idx]
            class_id = int(detection[5]) if len(detection) > 5 else -1
            self.tracks.append(Track(detection, class_id))
        
        # Remove dead tracks
        self.tracks = [t for t in self.tracks if not t.is_deleted()]
        
        # Get results
        results = []
        for track in self.tracks:
            if track.is_confirmed():
                box = track.get_state()
                results.append(np.append(box, [track.id, track.class_id, track.confidence]))
        
        return np.array(results) if results else np.empty((0, 7))
    
    def _match(self, detections):
        """Match detections to tracks using IoU"""
        if len(self.tracks) == 0 or len(detections) == 0:
            return [], list(range(len(self.tracks))), list(range(len(detections)))
        
        # Compute IoU matrix
        iou_matrix = np.zeros((len(self.tracks), len(detections)))
        for i, track in enumerate(self.tracks):
            for j, detection in enumerate(detections):
                iou_matrix[i, j] = self._iou(track.get_state(), detection[:4])
        
        # Convert to cost matrix
        cost_matrix = 1 - iou_matrix
        
        # Solve linear assignment problem
        row_indices, col_indices = lap.lapjv(cost_matrix, extend_cost=True)[0:2]
        
        # Filter matches
        matches = []
        unmatched_tracks = []
        unmatched_detections = []
        
        for track_idx, detection_idx in enumerate(col_indices):
            if detection_idx >= 0:
                if cost_matrix[track_idx, detection_idx] > self.max_iou_distance:
                    unmatched_tracks.append(track_idx)
                    unmatched_detections.append(detection_idx)
                else:
                    matches.append((track_idx, detection_idx))
            else:
                unmatched_tracks.append(track_idx)
        
        # Add unmatched detections
        for detection_idx in range(len(detections)):
            if detection_idx not in col_indices:
                unmatched_detections.append(detection_idx)
        
        return matches, unmatched_tracks, unmatched_detections
    
    def _iou(self, box1, box2):
        """Calculate IoU between two boxes"""
        # Convert to [x1, y1, x2, y2] if necessary
        box1 = np.array(box1).reshape(-1)
        box2 = np.array(box2).reshape(-1)
        
        # Calculate intersection area
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        intersection = max(0, x2 - x1) * max(0, y2 - y1)
        
        # Calculate union area
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        
        union = area1 + area2 - intersection
        
        # Calculate IoU
        return intersection / union if union > 0 else 0


class ObjectTracker:
    """Object tracker using YOLOv8 and Simple DeepSORT"""
    def __init__(self, model_path):
        """Initialize tracker with YOLOv8 model"""
        # Load YOLOv8 model
        self.model = YOLO(model_path)
        
        # Initialize tracker
        self.tracker = SimpleTracker()
        
        # Class names for VisDrone dataset
        self.class_names = ['pedestrian', 'people', 'bicycle', 'car', 'van', 
                           'truck', 'tricycle', 'awning-tricycle', 'bus', 'motor']
        
        # Dictionary to store trajectory points for each object
        self.trajectories = {}
        
        # Dictionary to store unique colors for each track ID
        self.track_colors = {}
    
    def _get_color_for_track(self, track_id):
        """Get a unique color for a track ID"""
        if track_id not in self.track_colors:
            # Generate a random color that's not too dark
            color = np.random.randint(80, 255, size=3).tolist()
            self.track_colors[track_id] = color
        return self.track_colors[track_id]
    
    def process_sequence(self, sequence_path, output_path, conf_threshold=0.3):
        """Process image sequence and create tracking video"""
        # Get all image files
        image_files = sorted([f for f in os.listdir(sequence_path) 
                             if f.endswith(('.jpg', '.png', '.jpeg'))])
        
        if not image_files:
            print(f"No images found in {sequence_path}")
            return
        
        # Read first image to get dimensions
        first_image = cv2.imread(os.path.join(sequence_path, image_files[0]))
        height, width = first_image.shape[:2]
        
        # Create output directory
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        # Initialize video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, 30, (width, height))
        
        # Process each frame
        for img_file in tqdm(image_files, desc="Processing frames"):
            # Read image
            img_path = os.path.join(sequence_path, img_file)
            frame = cv2.imread(img_path)
            if frame is None:
                continue
            
            # Run YOLOv8 detection
            results = self.model(frame, conf=conf_threshold, verbose=False)
            
            # Prepare detections for tracker
            detections = []
            if results[0].boxes.data.shape[0] > 0:
                # Get detection data: [x1, y1, x2, y2, conf, class_id]
                boxes = results[0].boxes.data.cpu().numpy()
                
                # Format detections for tracker
                for box in boxes:
                    x1, y1, x2, y2 = box[:4]
                    conf = box[4]
                    class_id = int(box[5]) if len(box) > 5 else 0
                    detections.append([x1, y1, x2, y2, conf, class_id])
            
            # Update tracker
            if detections:
                tracks = self.tracker.update(np.array(detections))
                
                # First draw trajectories
                for track in tracks:
                    # Extract tracking info
                    x1, y1, x2, y2, track_id, class_id, conf = track
                    
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    track_id = int(track_id)
                    
                    # Calculate center point of bounding box
                    center_x = int((x1 + x2) / 2)
                    center_y = int((y1 + y2) / 2)
                    
                    # Get unique color for this track ID
                    color = self._get_color_for_track(track_id)
                    
                    # Add center point to trajectory
                    if track_id not in self.trajectories:
                        self.trajectories[track_id] = []
                    self.trajectories[track_id].append((center_x, center_y))
                    
                    # Draw trajectory line
                    if len(self.trajectories[track_id]) > 1:
                        points = np.array(self.trajectories[track_id], np.int32)
                        cv2.polylines(frame, [points], False, color, 2)
                
                # Then draw bounding boxes (so they're on top of lines)
                for track in tracks:
                    # Extract tracking info
                    x1, y1, x2, y2, track_id, class_id, conf = track
                    
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    class_id = int(class_id)
                    track_id = int(track_id)
                    
                    # Get unique color for this track ID
                    color = self._get_color_for_track(track_id)
                    
                    # Get class name
                    class_name = self.class_names[class_id] if class_id < len(self.class_names) else 'unknown'
                    
                    # Draw bounding box
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    
                    # Draw class name and track ID
                    label = f"{class_name}-{track_id}"
                    t_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)[0]
                    cv2.rectangle(frame, (x1, y1-t_size[1]-10), (x1+t_size[0], y1), color, -1)
                    cv2.putText(frame, label, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
            
            # Write frame to output video
            out.write(frame)
            
            # Optional: Save each processed frame as an image
            output_img_dir = os.path.join(os.path.dirname(output_path), "processed_frames")
            os.makedirs(output_img_dir, exist_ok=True)
            cv2.imwrite(os.path.join(output_img_dir, img_file), frame)
        
        # Release video writer
        out.release()
        print(f"Output saved to {output_path}")
        print(f"Processed frames saved to {output_img_dir}")
        
        # Return output path
        return output_path

    def clear_trajectories(self):
        """Clear all tracked trajectories"""
        self.trajectories = {}
        self.track_colors = {}


# Function to process a test sequence
def process_test_sequence(model_path, sequence_path):
    """Process a test sequence with tracking"""
    # Initialize tracker
    tracker = ObjectTracker(model_path)
    
    # Output path
    os.makedirs("tracking_results", exist_ok=True)
    sequence_name = os.path.basename(sequence_path)
    output_path = f"tracking_results/{sequence_name}_tracked.mp4"
    
    # Process sequence
    result_path = tracker.process_sequence(sequence_path, output_path)
    
    print(f"Tracking completed! Video saved to: {result_path}")
    return result_path


# Main function
def main():
    # Path to your YOLOv8 model
    model_path = "yolov8n.pt"  # Update this with your model path
    
    # Direct path to the sequence folder
    sequence_path = "/kaggle/input/cv-multiobject-dectection-dataset/VisDrone2019-MOT-train/VisDrone2019-MOT-train/sequences/uav0000013_00000_v"
    
    # Process the sequence
    result_path = process_test_sequence(model_path, sequence_path)
    
    # Display information about the result
    print(f"Tracking results saved to: {result_path}")


if __name__ == "__main__":
    main()